In [99]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [100]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [101]:
customer = "Rosental"
from_date = '2025-10-21'
to_date = '2025-10-22'
goal_name = ["checkout_completed"]#,"signed_up"] #"checkout_completed"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [102]:
content={
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds(from_date),
                     "$lte": transform_date_to_timestamp_milliseconds(to_date),
               },
               "name":goal_name
                }
conversions=send_to_innkeepr_api_paginated(
    f"{url}/conversions/query",
    account_id,
    content,
    logging
)
conversions = pd.json_normalize(conversions)
conversions.columns

In [103]:
conversions

In [104]:
conversions["name"].value_counts()

In [105]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
# conversions = conversions[
#         conversions["signal.source"] == "609ffc4578188e83a2bc2c2c"
#     ]
print(conversions.shape)
vc = conversions.groupby("date")["name"].value_counts(dropna=False).reset_index()
vc.to_csv(f"DataChecks/conversions/conversions_{customer}.csv",index=False)
vc

In [106]:
conversions["properties.revenue"] = conversions["properties.revenue"].astype(float)
conversions.drop_duplicates(subset=["sessionId"]).groupby(by=["date"])["properties.revenue"].sum().reset_index()

In [107]:
conversions.groupby(by=["date","name","signal.source"])["anonymousId"].nunique().reset_index()

In [108]:
group = conversions.groupby("date")["signal.source"].value_counts(dropna=False).reset_index()
group

In [109]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
conversions["properties.revenue"] = conversions["properties.revenue"].astype(float)
group_rev = conversions.drop_duplicates(subset=["sessionId"]).groupby(by=["date","signal.source"])["properties.revenue"].sum().reset_index()
group_rev

In [110]:
conversions.groupby(by="date")["name"].count()

In [111]:
conversions.groupby(by="date")["anonymousId"].count()

In [112]:
conversions.groupby(by="date")["signal.source"].value_counts(dropna=False).reset_index()

In [113]:
conversions[(conversions["date"].astype("string")=="2025-09-25")&(conversions["signal.source"]=="609ffc4578188e83a2bc2c2c")]["sessionId"].unique()

In [114]:
conversions["signal.source"].value_counts()

In [115]:
conversions["created"].min(), conversions["created"].max()

In [116]:
conversions.drop_duplicates(subset=["name","sessionId"])["name"].value_counts(dropna=False)

In [117]:
conversions.drop_duplicates(subset=["name","anonymousId"])["name"].value_counts()

In [118]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
count_checkouts = conversions[conversions["name"].isin(goal_name)].drop_duplicates(subset=["name","sessionId"]).groupby("date")["name"].value_counts().reset_index().sort_values(by="date")
count_checkouts

In [119]:
# count_checkouts.plot(x="date",y="count",hue="name", kind="bar")
sns.barplot(data=count_checkouts, x="date",y="count",hue="name", palette="Set2")
plt.grid(True)
plt.xticks(rotation=90)

In [120]:
conversions["signal.source"].value_counts()

In [121]:
conversions[conversions["name"].isin(goal_name)].groupby(by=["signal.source","name"])["signal.relates_to.campaign.name"].value_counts().reset_index().sort_values(by="signal.relates_to.campaign.name")

In [122]:
conversions[conversions["name"].isin(goal_name)].groupby(by=["signal.source","name"])["signal.relates_to.treatment"].value_counts().reset_index()

In [123]:
conversions[conversions["name"].isin(goal_name)]["signal.relates_to.treatment"].unique().tolist()

In [124]:
vc_signal = conversions[conversions["name"].isin(goal_name)].groupby(by=["date","signal.source","name"])["signal.relates_to.campaign.name"].value_counts().reset_index().sort_values(by="date")
vc_signal

In [125]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(1,1,1)
sns.barplot(data=vc_signal[vc_signal["signal.source"]=="609ffc4578188e83a2bc2c2c"].sort_values(by="date"), x="date",y="count",hue="signal.source", palette="Set2", ax=ax)
plt.grid(True)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [126]:
conversions["properties.products"]

In [127]:
conversions

# Conversions by Month

In [128]:
conversions["created"].min(), conversions["created"].max()


In [129]:
conversions["name"].value_counts(dropna=False)

In [130]:
conversions["month"] = pd.to_datetime(conversions["created"]).dt.month
conversions.groupby(by="month")["sessionId"].nunique()

# Match Conversion with Treatments

In [131]:
treatments=send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":conversions["signal.relates_to.treatment"].dropna().unique().tolist()},
    logging
)
treatments = pd.json_normalize(treatments)
treatments.columns

In [132]:
treatments = treatments.rename(columns={"id": "signal.relates_to.treatment", "name":"treatment_name"})

In [133]:
conversions_with_treatments = pd.merge(conversions, treatments, how="left", on="signal.relates_to.treatment")
conversions_with_treatments

In [134]:
conversions_with_certain_campaigns = conversions_with_treatments[conversions_with_treatments["relates_to.campaign.name"].str.contains("ASC").fillna(False)]
conversions_with_certain_campaigns = conversions_with_certain_campaigns[conversions_with_certain_campaigns["relates_to.campaign.name"].str.contains("TOFU").fillna(False)]
conversions_with_certain_campaigns["relates_to.campaign.name"].value_counts()

In [135]:
conversions_with_certain_campaigns["signal.relates_to.treatment"].value_counts()

In [136]:
conversions_with_certain_campaigns["date"] = pd.to_datetime(conversions_with_certain_campaigns["created_x"]).dt.date
conversions_with_certain_campaigns.groupby("date")["name"].value_counts().reset_index()

In [137]:
conversions_with_treatments["signal.source"].value_counts()

# Match with account_signals

In [138]:
conversions["sessionId"].dropna().unique().tolist()

In [139]:
sessions = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"sessionId":conversions["sessionId"].dropna().unique().tolist()},
    logging
)
sessions = pd.json_normalize(sessions)
sessions.columns

In [140]:
gclids = sessions["campaign.gclid"].dropna().unique().tolist()
gclids[0:10]

In [141]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":gclids},
    logging
)

signals = pd.json_normalize(signals)
signals

In [142]:
adAccountId = "5193858957"
mask = signals["relates_to.adGroupAd.resourceName"].astype("str").str.contains(adAccountId)
signals = signals.loc[mask, ["relates_to.adGroupAd.resourceName", "externalId"]]
signals

In [143]:
merged = pd.merge(signals, sessions, how="left", left_on="externalId", right_on="campaign.gclid")
merged = pd.merge(merged, conversions, how="left", on="sessionId")
merged

In [144]:
merged.groupby("date")["sessionId"].nunique()


In [145]:
merged["properties.revenue"].astype("float").sum()


# Conversions with Google Gclids

In [146]:
check_for_anonymous_ids = ['68e60a081d063538d0dcb729', '64b562133991bf268073131d', '68e635bf1d063538d02ca781', '68e659381d063538d0644f2b', '64f8ba49f15d9306f00d9c4d', '68e6be97600b2140865b22d3', '68e5193c120f694f532d7643', '68e6d5a2600b2140868bca0e']
    
conversions[conversions["anonymousId"].isin(check_for_anonymous_ids)]

In [147]:
sessions = conversions["sessionId"].dropna().unique().tolist()
ses_resp = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"sessionId":sessions},
    logging
)
ses_resp = pd.json_normalize(ses_resp)
ses_resp.columns

In [148]:
ses_resp.groupby("sessionId")["anonymousId"].nunique()

In [149]:
conv_with_gclid = pd.merge(conversions, ses_resp[["sessionId","campaign.gclid"]], how="left", on="sessionId")
conv_with_gclid = conv_with_gclid[(conv_with_gclid["campaign.gclid"].notna()) &(conv_with_gclid["signal.source"].isnull()==False)]
#conv_with_gclid = conv_with_gclid[conv_with_gclid["created"]>="2025-10-08"]
#conv_with_gclid = conv_with_gclid[conv_with_gclid["created"]<="2025-10-09"]
conv_with_gclid

In [150]:
conv_with_gclid["sessionId"].nunique()

In [151]:
conv_with_gclid["anonymousId"].nunique()

In [152]:
profiles = send_to_innkeepr_api_paginated(
    f"{url}/profiles/query",
    account_id,
    {"anonymousId":conv_with_gclid["anonymousId"].unique().tolist()},
    logging
)
profiles = pd.json_normalize(profiles)
profiles = profiles.explode("externalIds")


In [153]:
profiles["has_gclid"] = profiles["externalIds"].apply(lambda x: x["name"]=="gclid")
profiles["extId"] = profiles["externalIds"].apply(lambda x: x["id"])
profiles = profiles[profiles["has_gclid"]==True]
profiles

In [154]:
print(conv_with_gclid["anonymousId"].unique().tolist())
for anomyousId in conv_with_gclid["anonymousId"].unique().tolist():
    if anomyousId not in profiles["anonymousId"].unique().tolist():
        print(anomyousId)

In [155]:
sessions = conversions["sessionId"].dropna().unique().tolist()
ses_resp_date = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"created":{
        "$gte":"2025-10-08",
        "$lt":"2025-10-09"
    }},
    logging
)
ses_resp_date = pd.json_normalize(ses_resp_date)
ses_resp_date.columns

In [156]:
ses_resp_date[ses_resp_date["sessionId"].isin(conv_with_gclid["sessionId"])]

# Ceck for E-Mail

In [157]:
for col in conversions.columns:
    temp = conversions[conversions[col].astype("string").str.contains("@|mail")]
    if len(temp) > 0:
        print(f"Found mail in {col}: {temp}")

In [158]:
test = [
    "_fbp",
    "_fbc",
    "email",
    "email_sha256",
    "anonymousId",
    "userAgent",
]
search_pattern = "|".join(test)